# Moving Average Crossover
In this notebook we will create a simple trading strategy using moving average crossover strategy. There are two kinds of signal we will be looking into for crossover strategy!
a. Slow Signal: Moving average of relatively longer time period
b. Fast Signal: Moving average of relatively shorter time period

example:

1. M.A.(10 days) is fast signal when compared with M.A.(50 days)

2. M.A.(50 days) is fast signal when compared with M.A.(200 days)

If fast signal cross over slow signal and is higher, we will buy the stock.
If slow signal goes above fast signal we will sell the stock. 

Note: If the fast signal is already above the slow signal, we won't enter the trade!

In [19]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline



In [44]:
aapl=yf.download("AAPL",start_date="2015-01-01", end_date="2020-01-01")
aapl.head(5)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600


# Step 1: Adding fast and slow signal into the dataframe !
Note: We will be using 10 period and 50 period moving average as fast and slow signal

In [45]:
aapl['MA10'] = aapl['Close'].rolling(10).mean()
aapl['MA50'] = aapl['Close'].rolling(50).mean()
aapl.head(5)

,Open,High,Low,Close,Adj Close,Volume,MA10,MA50
Date,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600,NaN,NaN
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800,NaN,NaN
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000,NaN,NaN
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600,NaN,NaN
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600,NaN,NaN


In [46]:
#Step 2: Dropping all NaN values rows from MA10 and MA50
aapl = aapl.dropna()
aapl.head(5)

,Open,High,Low,Close,Adj Close,Volume,MA10,MA50
Date,,,,,,,,
1981-02-24,0.107143,0.107143,0.106027,0.106027,0.082756,16979200,0.114676,0.132578
1981-02-25,0.112723,0.113281,0.112723,0.112723,0.087983,19488000,0.113783,0.132266
1981-02-26,0.114397,0.114955,0.114397,0.114397,0.089289,10841600,0.113448,0.132120
1981-02-27,0.118304,0.119420,0.118304,0.118304,0.092339,14761600,0.113616,0.132232
1981-03-02,0.118862,0.119420,0.118862,0.118862,0.092774,11760000,0.114118,0.132299


# Step 3 : Add a new column "Shares".
#If MA10>MA50, denote as 1 (long one share of stock), otherwise, denote as 0 (do nothing)


In [47]:
aapl['Shares'] = [1 if aapl.loc[ei, 'MA10']>aapl.loc[ei, 'MA50'] else 0 for ei in aapl.index]
aapl.head()

C:\Users\ibast\AppData\Local\Temp/ipykernel_112024/2560320147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['Shares'] = [1 if aapl.loc[ei, 'MA10']>aapl.loc[ei, 'MA50'] else 0 for ei in aapl.index]


,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares
Date,,,,,,,,,
1981-02-24,0.107143,0.107143,0.106027,0.106027,0.082756,16979200,0.114676,0.132578,0
1981-02-25,0.112723,0.113281,0.112723,0.112723,0.087983,19488000,0.113783,0.132266,0
1981-02-26,0.114397,0.114955,0.114397,0.114397,0.089289,10841600,0.113448,0.132120,0
1981-02-27,0.118304,0.119420,0.118304,0.118304,0.092339,14761600,0.113616,0.132232,0
1981-03-02,0.118862,0.119420,0.118862,0.118862,0.092774,11760000,0.114118,0.132299,0


In [49]:
#Step 4 : Restrucuring such that if shares == 1 then edit dataframe !!
#Note: If the fast signal is already above the slow signal, we will cut the dataframe such that the starting point is when Shares == 0
if aapl.iloc[0,-1] == 1:
    for i in range(len(aapl)):
        if aapl.iloc[i, -1] == 0:
            aapl = aapl.iloc[i:, :]
            break
            
aapl.head(2)

,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares
Date,,,,,,,,,
1981-02-24,0.107143,0.107143,0.106027,0.106027,0.082756,16979200,0.114676,0.132578,0
1981-02-25,0.112723,0.113281,0.112723,0.112723,0.087983,19488000,0.113783,0.132266,0


# Step 5 : Calculating Profit !!
Note: Profit is 0 if Shares = 0, i.e we aren't involved in any trading once Fast signal is below Slow signal!


In [50]:
aapl['Close1'] = aapl['Close'].shift(-1)
aapl['Profit'] = [aapl.loc[ei, 'Close1'] - aapl.loc[ei, 'Close'] if aapl.loc[ei, 'Shares']==1 else 0 for ei in aapl.index]
aapl.tail(2)

C:\Users\ibast\AppData\Local\Temp/ipykernel_112024/3004443108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['Close1'] = aapl['Close'].shift(-1)
C:\Users\ibast\AppData\Local\Temp/ipykernel_112024/3004443108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['Profit'] = [aapl.loc[ei, 'Close1'] - aapl.loc[ei, 'Close'] if aapl.loc[ei, 'Shares']==1 else 0 for ei in aapl.index]


,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares,Close1,Profit
Date,,,,,,,,,,,
2022-06-02,147.830002,151.270004,146.860001,151.210007,151.210007,72348100,144.111002,159.586401,0,145.380005,0.0
2022-06-03,146.899994,147.970001,144.460007,145.380005,145.380005,88471400,144.914001,159.089801,0,NaN,0.0


# Step 6 : Calculating Wealth
Wealth is cumsum of profit

In [51]:
aapl['wealth'] = aapl['Profit'].cumsum()
aapl.tail(3)

C:\Users\ibast\AppData\Local\Temp/ipykernel_112024/4137308804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['wealth'] = aapl['Profit'].cumsum()


,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares,Close1,Profit,wealth
Date,,,,,,,,,,,,
2022-06-01,149.899994,151.740005,147.679993,148.710007,148.710007,74286600,143.072002,159.938601,0,151.210007,0.0,93.557221
2022-06-02,147.830002,151.270004,146.860001,151.210007,151.210007,72348100,144.111002,159.586401,0,145.380005,0.0,93.557221
2022-06-03,146.899994,147.970001,144.460007,145.380005,145.380005,88471400,144.914001,159.089801,0,NaN,0.0,93.557221


# Step 7 : Calculating Buy price and net profit
Note: Approximation: That we bought at the highest price on the day we purchased ! As it's daily price data and we don't know exactly at what price did we bought. So to be at safe side, we consider the highest price of purchase during that day. 

In [52]:
buy_price = list(aapl[aapl['Shares'] == 1]['High'])[0]
net_profit = aapl.loc[aapl.index[-2], 'wealth']
profit_ratio = (net_profit/buy_price) * 100
print('Buy price: ${}, Net Profit: ${}, Profit Ratio: {}%'.format(buy_price, net_profit, round(profit_ratio, 3)))

Buy price: $0.125, Net Profit: $93.55722141638398, Profit Ratio: 74845.777%
